In [0]:
import random
import pandas as pd
import numpy as np
import sklearn
import math
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, LassoCV

In [0]:
test_data = pd.read_csv('test_data_full.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Replace the 'Unknown' to NA so as to make it recognizable as a missing value.

In [0]:
missing="Unknown"
testing_data=pd.read_csv('test_data_full.csv',na_values = missing)

Drop the variables that are not required from the start.

In [0]:
testing_data_copy=testing_data.drop(['Policy_Zip_Code_Garaging_Location','EEA_Policy_Zip_Code_3','EEA_PolicyYear','Vehicle_Make_Description'], axis=1)

In [0]:
testing_data_copy.shape

(985168, 62)

Make bins for Policy Number.

In [0]:
testing_data_copy['PolicyNo_<210']=testing_data_copy.apply(lambda _: '', axis=1)
testing_data_copy['PolicyNo_210-300']=testing_data_copy.apply(lambda _: '', axis=1)
testing_data_copy['PolicyNo_300-345']=testing_data_copy.apply(lambda _: '', axis=1)
testing_data_copy['PolicyNo_>345']=testing_data_copy.apply(lambda _: '', axis=1)

In [0]:
for i in testing_data_copy.index:

  record = testing_data_copy.get_value(i,'PolicyNo')

  if record <210000000:
     testing_data_copy.at[i,'PolicyNo_<210']=1
  else:
     testing_data_copy.at[ i,'PolicyNo_<210']=0

  if record >= 210000000 and record<300000000:
     testing_data_copy.at[ i,'PolicyNo_210-300']=1
  else:
     testing_data_copy.at[ i,'PolicyNo_210-300']=0

  if record >=300000000 and record<345000000:
     testing_data_copy.at[ i,'PolicyNo_300-345']=1
  else:
     testing_data_copy.at[ i,'PolicyNo_300-345']=0

  if record >=345000000:
     testing_data_copy.at[ i,'PolicyNo_>345']=1
  else:
     testing_data_copy.at[ i,'PolicyNo_>345']=0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
testing_data_copy.shape

(985168, 66)

Encode the data using get_dummies().

In [0]:
encod_test = pd.get_dummies(testing_data_copy,dummy_na=False, columns=['Policy_Company',
 'Policy_Installment_Term',
 'Policy_Billing_Code',
 'Policy_Method_Of_Payment',
 'Policy_Reinstatement_Fee_Indicator',
 'Vehicle_Territory',
 'Vehicle_Performance',
 'Vehicle_Symbol',
 'Vehicle_Usage',
 'Vehicle_Days_Per_Week_Driven',
 'Vehicle_Anti_Theft_Device',
 'Vehicle_Passive_Restraint',
 'Vehicle_Age_In_Years',
 'Vehicle_Med_Pay_Limit',
 'Vehicle_Bodily_Injury_Limit',
 'Vehicle_Physical_Damage_Limit',
 'Vehicle_Comprehensive_Coverage_Indicator',
 'Vehicle_Collision_Coverage_Indicator',
 'Vehicle_Collision_Coverage_Deductible',
 'Vehicle_Youthful_Driver_Indicator', 
 'Vehicle_Youthful_Driver_Training_Code',
 'Vehicle_Youthful_Good_Student_Code',
 'Vehicle_Driver_Points',
 'Vehicle_Safe_Driver_Discount_Indicator',
 'EEA_Liability_Coverage_Only_Indicator',
 'EEA_Multi_Auto_Policies_Indicator',
 'EEA_Agency_Type',
 'EEA_Packaged_Policy_Indicator',
 'EEA_Full_Coverage_Indicator',
 'EEA_Prior_Bodily_Injury_Limit',
 'SYS_Renewed',
 'SYS_New_Business',
 ])

In [0]:
encod_test.shape

(985168, 201)

Create new variables to be added to the encod_test data.

In [0]:
encod_test['Vehicle_Bodily_Injury_Limit_100-500_new']=encod_test['Vehicle_Bodily_Injury_Limit_100-300']+encod_test['Vehicle_Bodily_Injury_Limit_250-500']+encod_test['Vehicle_Bodily_Injury_Limit_300-500']+encod_test['Vehicle_Bodily_Injury_Limit_500-500']+encod_test['Vehicle_Bodily_Injury_Limit_100-500']
encod_test['Vehicle_Bodily_Injury_Limit_25-100_new']=encod_test['Vehicle_Bodily_Injury_Limit_25-50  ']+encod_test['Vehicle_Bodily_Injury_Limit_50-100 ']

encod_test['Vehicle_Make_Age']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Make_Age']=abs(2009-encod_test['Vehicle_Make_Year'])
encod_test['Vehicle_Make_Age_0-5']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Make_Age_5-10']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Make_Age_10-20']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Make_Age_20-50']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Make_Age_>50']=encod_test.apply(lambda _: '', axis=1)

for i in encod_test.index:

  record = encod_test.get_value(i,'Vehicle_Make_Age')

  if record >= 0 and record<5:
     encod_test.at[i,'Vehicle_Make_Age_0-5']=1
  else:
     encod_test.at[ i,'Vehicle_Make_Age_0-5']=0

  if record >= 5 and record<10:
     encod_test.at[ i,'Vehicle_Make_Age_5-10']=1
  else:
     encod_test.at[ i,'Vehicle_Make_Age_5-10']=0

  if record >=10 and record<20:
     encod_test.at[ i,'Vehicle_Make_Age_10-20']=1
  else:
     encod_test.at[ i,'Vehicle_Make_Age_10-20']=0

  if record >=20 and record<50:
     encod_test.at[ i,'Vehicle_Make_Age_20-50']=1
  else:
     encod_test.at[ i,'Vehicle_Make_Age_20-50']=0

  if record >=50:
     encod_test.at[ i,'Vehicle_Make_Age_>50']=1
  else:
     encod_test.at[ i,'Vehicle_Make_Age_>50']=0
     
encod_test['Vehicle_Number_Of_Drivers_Assigned_1']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Number_Of_Drivers_Assigned_2-8']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Number_Of_Drivers_Assigned_99']=encod_test.apply(lambda _: '', axis=1)

for i in encod_test.index:

  record = encod_test.get_value(i,'Vehicle_Number_Of_Drivers_Assigned')

  if record == 1:
    encod_test.at[ i,'Vehicle_Number_Of_Drivers_Assigned_1']=1
  else:
    encod_test.at[ i,'Vehicle_Number_Of_Drivers_Assigned_1']=0

  if record >=2 and record<9:
    encod_test.at[i,'Vehicle_Number_Of_Drivers_Assigned_2-8']=1
  else:
    encod_test.at[i,'Vehicle_Number_Of_Drivers_Assigned_2-8']=0

  if record >= 99:
    encod_test.at[i,'Vehicle_Number_Of_Drivers_Assigned_99']=1
  else:
    encod_test.at[i,'Vehicle_Number_Of_Drivers_Assigned_99']=0

encod_test['EEA_Policy_Tenure_0-10']=encod_test.apply(lambda _: '', axis=1)
encod_test['EEA_Policy_Tenure_10-20']=encod_test.apply(lambda _: '', axis=1)
encod_test['EEA_Policy_Tenure_20-30']=encod_test.apply(lambda _: '', axis=1)
encod_test['EEA_Policy_Tenure_30-40']=encod_test.apply(lambda _: '', axis=1)
encod_test['EEA_Policy_Tenure_40-50']=encod_test.apply(lambda _: '', axis=1)
encod_test['EEA_Policy_Tenure_50-60']=encod_test.apply(lambda _: '', axis=1)

for i in encod_test.index:

  record = encod_test.get_value(i,'EEA_Policy_Tenure')

  if record >=0 and record < 10 :
    encod_test.at[i,'EEA_Policy_Tenure_0-10']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_0-10']=0

  if record >=10 and record < 20 :
    encod_test.at[i,'EEA_Policy_Tenure_10-20']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_10-20']=0
  
  if record >=20 and record < 30 :
    encod_test.at[i,'EEA_Policy_Tenure_20-30']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_20-30']=0
  
  if record >=30 and record < 40 :
    encod_test.at[i,'EEA_Policy_Tenure_30-40']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_30-40']=0
  
  if record >=40 and record < 50 :
    encod_test.at[i,'EEA_Policy_Tenure_40-50']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_40-50']=0
  
  if record >=50 and record < 60 :
    encod_test.at[i,'EEA_Policy_Tenure_50-60']=1
  else:
    encod_test.at[i,'EEA_Policy_Tenure_50-60']=0
    
encod_test['Annual_Premium_0-1000']=encod_test.apply(lambda _: '', axis=1)
encod_test['Annual_Premium_1000-2000']=encod_test.apply(lambda _: '', axis=1)
encod_test['Annual_Premium_2000-3000']=encod_test.apply(lambda _: '', axis=1)
encod_test['Annual_Premium_3000-4000']=encod_test.apply(lambda _: '', axis=1)

for i in encod_test.index:
  
  record = encod_test.get_value(i,'Annual_Premium')

  if record >=0 and record < 1000 :
    encod_test.at[i,'Annual_Premium_0-1000']=1
  else:
    encod_test.at[i,'Annual_Premium_0-1000']=0

  if record >=1000 and record < 2000 :
    encod_test.at[i,'Annual_Premium_1000-2000']=1
  else:
    encod_test.at[i,'Annual_Premium_1000-2000']=0

  if record >=2000 and record < 3000 :
    encod_test.at[i,'Annual_Premium_2000-3000']=1
  else:
    encod_test.at[i,'Annual_Premium_2000-3000']=0

  if record >=3000:
    encod_test.at[i,'Annual_Premium_3000-4000']=1
  else:
    encod_test.at[i,'Annual_Premium_3000-4000']=0


encod_test['Vehicle_Miles_To_Work_0-10']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_10-20']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_20-30']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_30-40']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_40-50']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_50-60']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_60-70']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_70-80']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_80-90']=encod_test.apply(lambda _: '', axis=1)
encod_test['Vehicle_Miles_To_Work_90-100']=encod_test.apply(lambda _: '', axis=1)

for i in encod_test.index:

  record = encod_test.get_value(i,'Vehicle_Miles_To_Work')

  if record >=0 and record < 10 :
    encod_test.at[i,'Vehicle_Miles_To_Work_0-10']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_0-10']=0

  if record >=10 and record < 20 :
    encod_test.at[i,'Vehicle_Miles_To_Work_10-20']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_10-20']=0
  
  if record >=20 and record < 30 :
    encod_test.at[i,'Vehicle_Miles_To_Work_20-30']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_20-30']=0
  
  if record >=30 and record < 40 :
    encod_test.at[i,'Vehicle_Miles_To_Work_30-40']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_30-40']=0
  
  if record >=40 and record < 50 :
    encod_test.at[i,'Vehicle_Miles_To_Work_40-50']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_40-50']=0
  
  if record >=50 and record < 60 :
    encod_test.at[i,'Vehicle_Miles_To_Work_50-60']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_50-60']=0

  if record >=60 and record < 70 :
    encod_test.at[i,'Vehicle_Miles_To_Work_60-70']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_60-70']=0

  if record >=70 and record < 80 :
    encod_test.at[i,'Vehicle_Miles_To_Work_70-80']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_70-80']=0

  if record >=80 and record < 90 :
    encod_test.at[i,'Vehicle_Miles_To_Work_80-90']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_80-90']=0

  if record >=90 and record < 100 :
    encod_test.at[i,'Vehicle_Miles_To_Work_90-100']=1
  else:
    encod_test.at[i,'Vehicle_Miles_To_Work_90-100']=0    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:148: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [0]:
encod_test.shape

(985168, 232)

In [0]:
list(encod_test.columns)

['Unnamed: 0',
 'Portfolio',
 'PolicyNo',
 'Vehicle_New_Cost_Amount',
 'Vehicle_Number_Of_Drivers_Assigned',
 'Vehicle_Miles_To_Work',
 'Vehicle_Annual_Miles',
 'Vehicle_Comprehensive_Coverage_Limit',
 'Driver_Total',
 'Driver_Total_Male',
 'Driver_Total_Female',
 'Driver_Total_Single',
 'Driver_Total_Married',
 'Driver_Total_Related_To_Insured_Self',
 'Driver_Total_Related_To_Insured_Spouse',
 'Driver_Total_Related_To_Insured_Child',
 'Driver_Total_Licensed_In_State',
 'Driver_Minimum_Age',
 'Driver_Maximum_Age',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'EEA_Policy_Tenure',
 'Annual_Premium',
 'PolicyNo_<210',
 'PolicyNo_210-300',
 'PolicyNo_300-345',
 'PolicyNo_>345',
 'Policy_Company_Preferred',
 'Policy_Company_Stand

Drop 'Vehicle_Make_Year' as not needed anymore.

In [0]:
encod_test = encod_test.drop(['Vehicle_Make_Year'], axis=1)

In [0]:
encod_test.shape

(985168, 231)

In [0]:
# list(encod_test.columns)

In [0]:
encod_test = encod_test.drop(['PolicyNo'],axis=1)

In [0]:
encod_test.shape

(985168, 230)

Reorder encod_test df after adding all the new variables.

In [0]:
encod_test_temp=encod_test[['PolicyNo_<210',
 'PolicyNo_210-300',
 'PolicyNo_300-345',
 'PolicyNo_>345',
 'Policy_Company_Preferred',
 'Policy_Company_Standard ',
 'Policy_Installment_Term_6',
 'Policy_Installment_Term_12',
 'Policy_Billing_Code_Direct Billed to Insured',
 'Policy_Billing_Code_Premium Finance',
 'Policy_Method_Of_Payment_Installment',
 'Policy_Method_Of_Payment_Pre-paid',
 'Policy_Reinstatement_Fee_Indicator_N',
 'Policy_Reinstatement_Fee_Indicator_Y',
 'Vehicle_Territory_13',
 'Vehicle_Territory_15',
 'Vehicle_Territory_16',
 'Vehicle_Territory_17',
 'Vehicle_Territory_18',
 'Vehicle_Territory_19',
 'Vehicle_Territory_20',
 'Vehicle_Territory_22',
 'Vehicle_Territory_26',
 'Vehicle_Territory_27',
 'Vehicle_Territory_30',
 'Vehicle_Territory_31',
 'Vehicle_Territory_32',
 'Vehicle_Territory_35',
 'Vehicle_Territory_36',
 'Vehicle_Territory_37',
 'Vehicle_Make_Age',
 'Vehicle_Make_Age_0-5',
 'Vehicle_Make_Age_5-10',
 'Vehicle_Make_Age_10-20',
 'Vehicle_Make_Age_20-50',
 'Vehicle_Make_Age_>50',
 'Vehicle_Performance_High',
 'Vehicle_Performance_Intermediate',
 'Vehicle_Performance_Sports',
 'Vehicle_Performance_Sports Premium',
 'Vehicle_Performance_Standard',
 'Vehicle_New_Cost_Amount',
 'Vehicle_Symbol_-1',
 'Vehicle_Symbol_1',
 'Vehicle_Symbol_2',
 'Vehicle_Symbol_3',
 'Vehicle_Symbol_4',
 'Vehicle_Symbol_5',
 'Vehicle_Symbol_6',
 'Vehicle_Symbol_7',
 'Vehicle_Symbol_8',
 'Vehicle_Symbol_9',
 'Vehicle_Symbol_10',
 'Vehicle_Symbol_11',
 'Vehicle_Symbol_12',
 'Vehicle_Symbol_13',
 'Vehicle_Symbol_14',
 'Vehicle_Symbol_15',
 'Vehicle_Symbol_16',
 'Vehicle_Symbol_17',
 'Vehicle_Symbol_18',
 'Vehicle_Symbol_19',
 'Vehicle_Symbol_20',
 'Vehicle_Symbol_21',
 'Vehicle_Symbol_22',
 'Vehicle_Symbol_23',
 'Vehicle_Symbol_24',
 'Vehicle_Symbol_25',
 'Vehicle_Symbol_26',
 'Vehicle_Symbol_27',
 'Vehicle_Number_Of_Drivers_Assigned',
 'Vehicle_Number_Of_Drivers_Assigned_1',
 'Vehicle_Number_Of_Drivers_Assigned_2-8',
 'Vehicle_Number_Of_Drivers_Assigned_99',
 'Vehicle_Usage_Business',
 'Vehicle_Usage_Farm    ',
 'Vehicle_Usage_Pleasure',
 'Vehicle_Usage_Work    ',
 'Vehicle_Miles_To_Work',
 'Vehicle_Miles_To_Work_0-10',
 'Vehicle_Miles_To_Work_10-20',
 'Vehicle_Miles_To_Work_20-30',
 'Vehicle_Miles_To_Work_30-40',
 'Vehicle_Miles_To_Work_40-50',
 'Vehicle_Miles_To_Work_50-60',
 'Vehicle_Miles_To_Work_60-70',
 'Vehicle_Miles_To_Work_70-80',
 'Vehicle_Miles_To_Work_80-90',
 'Vehicle_Miles_To_Work_90-100',
 'Vehicle_Days_Per_Week_Driven_-1',
 'Vehicle_Days_Per_Week_Driven_0',
 'Vehicle_Days_Per_Week_Driven_1',
 'Vehicle_Days_Per_Week_Driven_2',
 'Vehicle_Days_Per_Week_Driven_3',
 'Vehicle_Days_Per_Week_Driven_4',
 'Vehicle_Days_Per_Week_Driven_5',
 'Vehicle_Days_Per_Week_Driven_6',
 'Vehicle_Days_Per_Week_Driven_7',
 'Vehicle_Annual_Miles',
 'Vehicle_Anti_Theft_Device_Active Disabling',
 'Vehicle_Anti_Theft_Device_Alarm Only',
 'Vehicle_Anti_Theft_Device_Not Applicable',
 'Vehicle_Anti_Theft_Device_Passive Disabling-Vehicle Recovery',
 'Vehicle_Passive_Restraint_N',
 'Vehicle_Passive_Restraint_Y',
 'Vehicle_Age_In_Years_1',
 'Vehicle_Age_In_Years_2',
 'Vehicle_Age_In_Years_3',
 'Vehicle_Age_In_Years_4',
 'Vehicle_Age_In_Years_5',
 'Vehicle_Age_In_Years_6',
 'Vehicle_Age_In_Years_7',
 'Vehicle_Age_In_Years_8',
 'Vehicle_Age_In_Years_9',
 'Vehicle_Med_Pay_Limit_-1',
 'Vehicle_Med_Pay_Limit_1000',
 'Vehicle_Med_Pay_Limit_2000',
 'Vehicle_Med_Pay_Limit_5000',
 'Vehicle_Med_Pay_Limit_10000',
 'Vehicle_Med_Pay_Limit_25000',
 'Vehicle_Med_Pay_Limit_50000',
 'Vehicle_Bodily_Injury_Limit_25-100_new',
 'Vehicle_Bodily_Injury_Limit_25-50  ',
 'Vehicle_Bodily_Injury_Limit_50-100 ',
 'Vehicle_Bodily_Injury_Limit_100-500',
 'Vehicle_Bodily_Injury_Limit_100-500_new',
 'Vehicle_Bodily_Injury_Limit_100-300',
 'Vehicle_Bodily_Injury_Limit_250-500',
 'Vehicle_Bodily_Injury_Limit_300-500',
 'Vehicle_Bodily_Injury_Limit_500-500',
 'Vehicle_Bodily_Injury_Limit_1M-1M  ',
 'Vehicle_Physical_Damage_Limit_-1',
 'Vehicle_Physical_Damage_Limit_25000',
 'Vehicle_Physical_Damage_Limit_35000',
 'Vehicle_Physical_Damage_Limit_50000',
 'Vehicle_Physical_Damage_Limit_100000',
 'Vehicle_Physical_Damage_Limit_250000',
 'Vehicle_Physical_Damage_Limit_500000',
 'Vehicle_Physical_Damage_Limit_1000000',
 'Vehicle_Comprehensive_Coverage_Indicator_N',
 'Vehicle_Comprehensive_Coverage_Indicator_Y',
 'Vehicle_Comprehensive_Coverage_Limit',
 'Vehicle_Collision_Coverage_Indicator_N',
 'Vehicle_Collision_Coverage_Indicator_Y',
 'Vehicle_Collision_Coverage_Deductible_-1',
 'Vehicle_Collision_Coverage_Deductible_50',
 'Vehicle_Collision_Coverage_Deductible_100',
 'Vehicle_Collision_Coverage_Deductible_200',
 'Vehicle_Collision_Coverage_Deductible_250',
 'Vehicle_Collision_Coverage_Deductible_500',
 'Vehicle_Collision_Coverage_Deductible_1000',
 'Vehicle_Collision_Coverage_Deductible_2000',
 'Vehicle_Collision_Coverage_Deductible_3000',
 'Driver_Total',
 'Driver_Total_Male',
 'Driver_Total_Female',
 'Driver_Total_Single',
 'Driver_Total_Married',
 'Driver_Total_Related_To_Insured_Self',
 'Driver_Total_Related_To_Insured_Spouse',
 'Driver_Total_Related_To_Insured_Child',
 'Driver_Total_Licensed_In_State',
 'Driver_Minimum_Age',
 'Driver_Maximum_Age',
 'Driver_Total_Teenager_Age_15_19',
 'Driver_Total_College_Ages_20_23',
 'Driver_Total_Young_Adult_Ages_24_29',
 'Driver_Total_Low_Middle_Adult_Ages_30_39',
 'Driver_Total_Middle_Adult_Ages_40_49',
 'Driver_Total_Adult_Ages_50_64',
 'Driver_Total_Senior_Ages_65_69',
 'Driver_Total_Upper_Senior_Ages_70_plus',
 'Vehicle_Youthful_Driver_Indicator_N',
 'Vehicle_Youthful_Driver_Indicator_Y',
 'Vehicle_Youthful_Driver_Training_Code_Not Applicable                 ',
 'Vehicle_Youthful_Driver_Training_Code_With Driver Training           ',
 'Vehicle_Youthful_Driver_Training_Code_With or Without Driver Training',
 'Vehicle_Youthful_Driver_Training_Code_Without Driver Training        ',
 'Vehicle_Youthful_Good_Student_Code_Eligible for Good Student Credit    ',
 'Vehicle_Youthful_Good_Student_Code_Not Eligible for Good Student Credit',
 'Vehicle_Driver_Points_0',
 'Vehicle_Driver_Points_1',
 'Vehicle_Driver_Points_2',
 'Vehicle_Driver_Points_3',
 'Vehicle_Driver_Points_4',
 'Vehicle_Driver_Points_5',
 'Vehicle_Driver_Points_6',
 'Vehicle_Driver_Points_7',
 'Vehicle_Safe_Driver_Discount_Indicator_N',
 'Vehicle_Safe_Driver_Discount_Indicator_Y',
 'EEA_Liability_Coverage_Only_Indicator_N',
 'EEA_Liability_Coverage_Only_Indicator_Y',
 'EEA_Multi_Auto_Policies_Indicator_N',
 'EEA_Multi_Auto_Policies_Indicator_Y',
 'EEA_Policy_Tenure',
 'EEA_Policy_Tenure_0-10',
 'EEA_Policy_Tenure_10-20',
 'EEA_Policy_Tenure_20-30',
 'EEA_Policy_Tenure_30-40',
 'EEA_Policy_Tenure_40-50',
 'EEA_Policy_Tenure_50-60',
 'EEA_Agency_Type_Hybrid',
 'EEA_Agency_Type_Non-standard',
 'EEA_Agency_Type_Preferred',
 'EEA_Agency_Type_Standard',
 'EEA_Packaged_Policy_Indicator_N',
 'EEA_Packaged_Policy_Indicator_Y',
 'EEA_Full_Coverage_Indicator_N',
 'EEA_Full_Coverage_Indicator_Y',
 'EEA_Prior_Bodily_Injury_Limit_100-200',
 'EEA_Prior_Bodily_Injury_Limit_100-400',
 'EEA_Prior_Bodily_Injury_Limit_20-50',
 'EEA_Prior_Bodily_Injury_Limit_200-400',
 'EEA_Prior_Bodily_Injury_Limit_300-300',
 'EEA_Prior_Bodily_Injury_Limit_40-100 ',
 'EEA_Prior_Bodily_Injury_Limit_75-300',
 'EEA_Prior_Bodily_Injury_Limit_750-750',
 'SYS_Renewed_N',
 'SYS_Renewed_Y',
 'SYS_New_Business_N',
 'SYS_New_Business_Y',
 'Annual_Premium',
 'Annual_Premium_0-1000',
 'Annual_Premium_1000-2000',
 'Annual_Premium_2000-3000',
 'Annual_Premium_3000-4000'
]]

In [0]:
list(encod_test_temp.columns)

['PolicyNo_<210',
 'PolicyNo_210-300',
 'PolicyNo_300-345',
 'PolicyNo_>345',
 'Policy_Company_Preferred',
 'Policy_Company_Standard ',
 'Policy_Installment_Term_6',
 'Policy_Installment_Term_12',
 'Policy_Billing_Code_Direct Billed to Insured',
 'Policy_Billing_Code_Premium Finance',
 'Policy_Method_Of_Payment_Installment',
 'Policy_Method_Of_Payment_Pre-paid',
 'Policy_Reinstatement_Fee_Indicator_N',
 'Policy_Reinstatement_Fee_Indicator_Y',
 'Vehicle_Territory_13',
 'Vehicle_Territory_15',
 'Vehicle_Territory_16',
 'Vehicle_Territory_17',
 'Vehicle_Territory_18',
 'Vehicle_Territory_19',
 'Vehicle_Territory_20',
 'Vehicle_Territory_22',
 'Vehicle_Territory_26',
 'Vehicle_Territory_27',
 'Vehicle_Territory_30',
 'Vehicle_Territory_31',
 'Vehicle_Territory_32',
 'Vehicle_Territory_35',
 'Vehicle_Territory_36',
 'Vehicle_Territory_37',
 'Vehicle_Make_Age',
 'Vehicle_Make_Age_0-5',
 'Vehicle_Make_Age_5-10',
 'Vehicle_Make_Age_10-20',
 'Vehicle_Make_Age_20-50',
 'Vehicle_Make_Age_>50',
 '

In [0]:
encod_test_temp.shape

(985168, 226)

In [0]:
#Save as csv

encod_test.to_csv('encod_test_first_wPolicyNo.csv', index=False)

Divide into portfolios.

In [0]:
val = encod_test['Portfolio'][0]
list_of_dataframes = []
portfolio=pd.DataFrame()
for i in range(len(encod_test)):
    if encod_test['Portfolio'][i] == val:
        portfolio=portfolio.append(encod_test.iloc[[i]])
    else:
        val = encod_test['Portfolio'][i]
        list_of_dataframes.append(portfolio)
        portfolio=pd.DataFrame()
        portfolio=portfolio.append(encod_test.iloc[[i]])
list_of_dataframes.append(portfolio)

Helper functions to summarize the portfolios.

In [0]:
#For finding sum
def portfolio_sum(portfolio,column_name):
  sum=0
  for record in portfolio[column_name]:
    if  (not np.isnan(record)) and record !=-1:
      sum+=record
  # print(sum)
  return sum
  
def summarize_portfolio(portfolio):
  portfolio = portfolio.drop(['Unnamed: 0', 'Portfolio'], axis=1)
  new_port=pd.DataFrame()
  # new_port=pd.DataFrame([dataframe['Portfolio'].iloc[0]], columns=['Portfolio'])
  #For finding percentage
  for record in portfolio:
    new_port[record]=new_port.apply(lambda _: '', axis=1)
    if set([0,1]).issubset(set(portfolio[record].value_counts(normalize=True).keys())) and len(portfolio[record].value_counts(normalize=True))==2:
        new_port.at[0,record]=round(portfolio[record].value_counts(normalize=True)[1]*100,2)
    if (portfolio[record]==0).all():
        new_port.at[0,record]=0
    if (portfolio[record]==1).all():
        new_port.at[0,record]=1
  #For finding mean and median
  new_port['Vehicle_Make_Age']=portfolio['Vehicle_Make_Age'].mean()
  new_port['Vehicle_Make_Age_Median']=portfolio['Vehicle_Make_Age'].median()
  new_port['Driver_Minimum_Age']=portfolio['Driver_Minimum_Age'].mean()
  new_port['Driver_Minimum_Age_Median']=portfolio['Driver_Minimum_Age'].median()
  new_port['Driver_Maximum_Age']=portfolio['Driver_Maximum_Age'].mean()
  new_port['Driver_Maximum_Age_Median']=portfolio['Driver_Maximum_Age'].median()
  #For finding sum
  new_port['Vehicle_New_Cost_Amount']=portfolio_sum(portfolio,'Vehicle_New_Cost_Amount')
  new_port['Vehicle_Number_Of_Drivers_Assigned']=portfolio_sum(portfolio,'Vehicle_Number_Of_Drivers_Assigned')
  new_port['Vehicle_Miles_To_Work']=portfolio_sum(portfolio,'Vehicle_Miles_To_Work')
  new_port['Vehicle_Annual_Miles']=portfolio_sum(portfolio,'Vehicle_Annual_Miles')
  new_port['Vehicle_Comprehensive_Coverage_Limit']=portfolio_sum(portfolio,'Vehicle_Comprehensive_Coverage_Limit')
  new_port['Driver_Total']=portfolio_sum(portfolio,'Driver_Total')
  new_port['Driver_Total_Male']=portfolio_sum(portfolio,'Driver_Total_Male')
  new_port['Driver_Total_Female']=portfolio_sum(portfolio,'Driver_Total_Female')
  new_port['Driver_Total_Single']=portfolio_sum(portfolio,'Driver_Total_Single')
  new_port['Driver_Total_Married']=portfolio_sum(portfolio,'Driver_Total_Married')
  new_port['Driver_Total_Related_To_Insured_Self']=portfolio_sum(portfolio,'Driver_Total_Related_To_Insured_Self')
  new_port['Driver_Total_Related_To_Insured_Spouse']=portfolio_sum(portfolio,'Driver_Total_Related_To_Insured_Spouse')
  new_port['Driver_Total_Related_To_Insured_Child']=portfolio_sum(portfolio,'Driver_Total_Related_To_Insured_Child')
  new_port['Driver_Total_Licensed_In_State']=portfolio_sum(portfolio,'Driver_Total_Licensed_In_State')
  new_port['Driver_Total_Teenager_Age_15_19']=portfolio_sum(portfolio,'Driver_Total_Teenager_Age_15_19')
  new_port['Driver_Total_College_Ages_20_23']=portfolio_sum(portfolio,'Driver_Total_College_Ages_20_23')
  new_port['Driver_Total_Young_Adult_Ages_24_29']=portfolio_sum(portfolio,'Driver_Total_Young_Adult_Ages_24_29')
  new_port['Driver_Total_Low_Middle_Adult_Ages_30_39']=portfolio_sum(portfolio,'Driver_Total_Low_Middle_Adult_Ages_30_39')
  new_port['Driver_Total_Middle_Adult_Ages_40_49']=portfolio_sum(portfolio,'Driver_Total_Middle_Adult_Ages_40_49')
  new_port['Driver_Total_Adult_Ages_50_64']=portfolio_sum(portfolio,'Driver_Total_Adult_Ages_50_64')
  new_port['Driver_Total_Senior_Ages_65_69']=portfolio_sum(portfolio,'Driver_Total_Senior_Ages_65_69')
  new_port['Driver_Total_Upper_Senior_Ages_70_plus']=portfolio_sum(portfolio,'Driver_Total_Upper_Senior_Ages_70_plus')
  new_port['EEA_Policy_Tenure']=portfolio_sum(portfolio,'EEA_Policy_Tenure')
  new_port['Annual_Premium']=portfolio_sum(portfolio,'Annual_Premium')
  return new_port

Summarize the portfolios and stored the portfolio names in an array.

In [0]:
summarized_dataframes=pd.DataFrame()
i=0
port_id=[]
for dataframe in list_of_dataframes:
  port_id.append(dataframe['Portfolio'].iloc[0])
  new=pd.DataFrame()
  # print(i)
  new=summarize_portfolio(dataframe)
  summarized_dataframes=summarized_dataframes.append(new)
  i+=1

Strip 'test_' and '.csv' from the portfolio ID

In [0]:
# port_id
for i in range (len(port_id)):
  port_id[i] = port_id[i].strip('.csv')
  port_id[i] = port_id[i].strip('test_')

In [0]:
Final = pd.DataFrame()
Final['ID'] = port_id

Final

Save Portfolio IDs to csv

In [0]:
Final.to_csv('Final.csv', index=False)